In [1]:
import pandas as pd
import random
import librosa
import os
from ketos.data_handling.parsing import load_audio_representation
from ketos.data_handling.database_interface import AudioWriter, create_database
from ketos.data_handling import selection_table as sl

2021-11-30 22:43:51.215915: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-30 22:43:51.621566: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10477 MB memory:  -> device: 0, name: NVIDIA TITAN V, pci bus id: 0000:a1:00.0, compute capability: 7.0
/home/sadman/anaconda3/envs/deeplearning/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


# Initialization

In [2]:
# Database-related parameters
db_name_string='CB300' # 'CB300' or 'CB50'
data_dir='/data/WCS/' # Path to the folder that contains all the audio files
dataset_name='/'+db_name_string

spec_cfg = load_audio_representation('spec_config.json', name="spectrogram")

# Load annotation files
root_path='/home/sadman/arctic_mammal/' # Root path to all code/annotation
annot_filename=db_name_string+'_train.csv'

db_save_path=root_path+'saved_database/'
db_save_filename='db_test_'+str(spec_cfg['duration'])+'.h5'

# If want to use all annotations, then keep False, otherwise if want to use annotations based on quality, then True
filter_based_on_quality_flag=False 

annot_df=pd.read_csv(root_path+'annotations/train/'+annot_filename, sep=',')

In [3]:
annot_df

,Selection,View,Channel,Begin Time (s),End Time (s),Delta Time (s),File Offset (s),Low Freq (Hz),High Freq (Hz),Peak Freq (Hz),...,AR comment,SNR NIST Quick (dB FS),SNR order,species,certainty,comment,type,WH ID,WH Comment,NEW analysis
0,4825,Spectrogram 1,1,43005.721333,43007.475333,1.754,117.248,47.6,470.2,93.8,...,NaN,40.43,217,BH,High,NaN,NaN,NaN,NaN,y
1,2503,Spectrogram 1,1,13132.109667,13134.101667,1.992,235.314,185.6,676.6,199.2,...,NaN,13.13,4192,BH,High,ds,ssong,NaN,NaN,yes
2,9342,Spectrogram 1,1,141645.687633,141646.945633,1.258,81.163,64.1,163.1,117.2,...,NaN,2.31,5593,BH,High,NaN,NaN,NaN,NaN,NaN
3,5491,Spectrogram 1,1,53861.496367,53863.120367,1.624,175.749,61.9,216.5,175.8,...,NaN,26.63,3198,BH,High,NaN,NaN,NaN,NaN,y
4,11636,Spectrogram 1,1,187384.257000,187392.701000,8.444,232.162,67.2,7258.7,228.5,...,NaN,13.17,4177,BH,High,NaN,roar,NaN,NaN,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4780,11567,Spectrogram 1,1,186586.066033,186588.556033,2.490,33.841,59.4,344.6,64.5,...,NaN,34.10,1809,BH,High,NaN,NaN,NaN,NaN,yes
4781,9162,Spectrogram 1,1,137259.443700,137260.214700,0.771,193.645,93.2,314.6,193.4,...,NaN,37.61,758,BH,High,faint,us,NaN,NaN,NaN
4782,5464,Spectrogram 1,1,53505.164967,53506.853967,1.689,119.268,61.9,364.9,199.2,...,NaN,31.88,2503,BH,High,NaN,NaN,NaN,NaN,y
4783,6813,Spectrogram 1,1,84310.844367,84312.140367,1.296,31.579,80.2,246.2,123.0,...,NaN,30.93,2748,BH,High,NaN,NaN,NaN,NaN,NaN


# Annotation file pre-processing

In [4]:
def delete_unnecessary_columns(annot_df, delete_columns=['Selection', 'View']):
    """ Delete unwanted columns from the annotations dataframe
     
        Args:
            annot_df: pandas DataFrame
                Annotation table.
            delete_columns: list
                List of columns to delete from the annotations dataframe
                default values ('Selection', 'View')

        Returns:
            annot_df: pandas DataFrame
                Annotation table after removing listed columns

    """
    for column in delete_columns:
        del annot_df[column]
    return annot_df

annot_df=delete_unnecessary_columns(annot_df)

annot_df.rename({'Begin Path': 'filename',
                'File Offset (s)': 'start',
                'species': 'label'}, axis='columns', inplace =True)

# Modify filepath (Discard the drive location (e.g., D:/))
annot_df['filename']=annot_df['filename'].apply(lambda x: x[3:len(x)]) 
# Modify filepath to replace \ with / in the filename
annot_df['filename']=annot_df['filename'].apply(lambda x: x.replace("\\", "/")) 
# Modify filepath to add the data root dir (/data/WCS/)
annot_df['filename']=annot_df['filename'].apply(lambda x: data_dir+x)
# Calculate End time
annot_df['end']=annot_df['start']+annot_df['Delta Time (s)']

# Add Negative samples

In [5]:
# Change the location of 'target_wav_location' to choose random neg samples from that location
list_of_wav_files=annot_df['filename'].unique()
print("list_of_wav_files: ", len(list_of_wav_files))

# Define the ratio of files to select from the folder
add_ratio=0.1
total_files=int(len(list_of_wav_files)*add_ratio)
print(total_files)

# Now, generate list of unique file indices
file_indices_random = random.sample(range(0, len(list_of_wav_files)), total_files)

list_of_wav_files:  813
81


In [6]:
# ========= Adding Negative (no BH) samples ==========
total_wav_parsed = 0
total_number_of_negative_samples=0

for file_index in file_indices_random:
    target_wav_filename=list_of_wav_files[file_index]
    # target_wav_filename=target_wav_filename[3:len(target_wav_filename)]
    target_matched_annot_df = annot_df[annot_df['filename'].str.find(os.path.basename(target_wav_filename)) != -1]
    
    if(len(target_matched_annot_df)!=0):
        print("target_wav_filename:", target_wav_filename)
        total_time_of_file = librosa.get_duration(filename=target_wav_filename)
        target_files_with_len = pd.DataFrame({'filename':[target_wav_filename], 
                                              'duration':[total_time_of_file]})

        print("Total annotations found: ", len(target_matched_annot_df))

        #Standardize annotation table format
        annot, label_dict = sl.standardize(target_matched_annot_df, return_label_dict=True, trim_table=True)


        sel = sl.select_by_segmenting(files=target_files_with_len, 
                                   length=spec_cfg['duration'], 
                                   annotations=annot, 
                                   step=spec_cfg['duration'],
                                   keep_only_empty=True) 

        print("sel:", len(sel))
        
        # Check if there is any BH class in the selection table
        sel_only_neg=sel.loc[sel['label']==0] 
        sel_only_pos=sel.loc[sel['label']==1]
        
        print("sel_only_pos:", len(sel_only_pos))
        print("sel_only_neg:", len(sel_only_neg))
        
        print("Adding samples to "+dataset_name)
        
        create_database(output_file=db_save_path+db_save_filename, data_dir='',
                                        dataset_name=dataset_name+'/Neg', selections=sel_only_neg,
                                        audio_repres=spec_cfg)

        total_wav_parsed+=1
        total_number_of_negative_samples+=len(sel_only_neg)

target_wav_filename: /data/WCS/Cape_Bathurst_300_2018_2019/1208533023.050316235015.wav
Total annotations found:  2
sel: 98
sel_only_pos: 0
sel_only_neg: 98
Adding samples to /CB300


100%|███████████████████████████████████████████| 98/98 [00:02<00:00, 37.30it/s]


98 items saved to /home/sadman/arctic_mammal/saved_database/db_test_3.0.h5
target_wav_filename: /data/WCS/Cape_Bathurst_300_2018_2019/1208533023.050109205015.wav
Total annotations found:  7
sel: 87
sel_only_pos: 0
sel_only_neg: 87
Adding samples to /CB300


100%|███████████████████████████████████████████| 87/87 [00:02<00:00, 37.81it/s]


87 items saved to /home/sadman/arctic_mammal/saved_database/db_test_3.0.h5
target_wav_filename: /data/WCS/Cape_Bathurst_300_2018_2019/1208533023.050112005015.wav
Total annotations found:  20
sel: 48
sel_only_pos: 0
sel_only_neg: 48
Adding samples to /CB300


...

96 items saved to /home/sadman/arctic_mammal/saved_database/db_test_3.0.h5
target_wav_filename: /data/WCS/Cape_Bathurst_300_2018_2019/1208533023.041029015015.wav
Total annotations found:  8
sel: 86
sel_only_pos: 0
sel_only_neg: 86
Adding samples to /CB300


100%|███████████████████████████████████████████| 86/86 [00:02<00:00, 38.56it/s]


86 items saved to /home/sadman/arctic_mammal/saved_database/db_test_3.0.h5
target_wav_filename: /data/WCS/Cape_Bathurst_300_2018_2019/1208533023.041125115015.wav
Total annotations found:  4
sel: 92
sel_only_pos: 0
sel_only_neg: 92
Adding samples to /CB300


100%|███████████████████████████████████████████| 92/92 [00:02<00:00, 37.64it/s]

92 items saved to /home/sadman/arctic_mammal/saved_database/db_test_3.0.h5


In [7]:
# Filter the annotations with 1/2/3 values in the Tonal, Signal strength, and Background columns
if(filter_based_on_quality_flag):
    annot_df=annot_df[annot_df['Tonal'].isin(['1', '2', '3'])]

# Add Positive (BH) samples

In [8]:
# ========= Adding Positive (BH) samples ==========
map_to_ketos_annot_std ={'Begin Path': 'filename',
                          'File Offset (s)': 'start',
                          'species': 'label'
                        } 
std_annot_BH_df = sl.standardize(table=annot_df, mapper=map_to_ketos_annot_std, 
                              signal_labels=['BH'], trim_table=True)
sel_table_BH = sl.select(annotations=std_annot_BH_df, length=spec_cfg['duration'], step=0.5, min_overlap=0.8, center=False)

In [9]:
create_database(output_file=db_save_path+db_save_filename, data_dir='',
                                dataset_name=dataset_name+'/BH',selections=std_annot_BH_df,
                                audio_repres=spec_cfg)

100%|███████████████████████████████████████| 4785/4785 [02:00<00:00, 39.61it/s]

4785 items saved to /home/sadman/arctic_mammal/saved_database/db_test_3.0.h5


In [10]:
print("Total negative samples saved: ", total_number_of_negative_samples)
print("Total BH samples saved (Before stepping augmentation): ", len(std_annot_BH_df))
print("Total BH samples saved (After stepping augmentation): ", len(sel_table_BH))

Total negative samples saved:  7308
Total BH samples saved (Before stepping augmentation):  4785
Total BH samples saved (After stepping augmentation):  8205
